In [1]:
import sys
import pickle
import numpy as np
import pandas as pd
import mml_lib.visualization as viz
sys.path.append('/home/jovyan/work/a091569/cython_home')
sys.path.append('/home/jovyan/work/a091569/cython_home/cython_home')

# cython utilities
from get_csv_rows import get_csv_rows
from csv_event_rows_to_dict import csv_event_rows_to_dict

In [3]:
!ls *.csv

product_attributes_csv_20191019.csv	  pros_cart_item_pur_csv_20191027.csv
pros_cart_item_addn_csv_120_20191019.csv  pros_prod_vw_csv_120_20191019.csv
pros_cart_item_addn_csv_20191019.csv	  results-20191022-152105.csv
pros_cart_item_pur_csv_120_20191019.csv


In [53]:
!head -n 10 pros_prod_vw_csv_120_pros_product_view.csv

user_indiv_id	itemid	evnt_ts	sessionid
161322486	1806	2019-06-26 17:46:22	57133676298705600281061543070534137125503
123152321	64887	2019-08-04 11:05:37	756201711834091504050373584764359394489975620171183409150405037358476435939448
126175897	111175	2019-07-15 18:55:43	850338558370763480082111563216276234052559
150319733	212226	2019-10-02 02:49:06	2372149009064601358465935340155052781232372149009064601358465935340155052781
277726415	212226	2019-07-03 18:01:45	159507908974240456882031562168544009886329
1038986511	212226	2019-10-15 20:39:27	3244989876172701418691754834801460080015932449898761727014186917548348014600800
149602045	212226	2019-07-18 03:01:01	530644917336061142530191322424243117940558
177000459	212226	2019-10-02 21:54:35	3299276396631141872461168715659373194519932992763966311418724611687156593731945
144607372	212226	2019-08-22 13:06:39	5195397003545820487476546375366780505205195397003545820487476546375366780505


In [54]:
%%load /home/jovyan/work/a091569/

/home/jovyan/work/a091569/DTOOLS/BIGQUERY_DATA


In [ ]:
%%time
view_dict = csv_event_rows_to_dict('pros_prod_vw_csv_120_pros_product_view.csv',chunksize=800000)
print(len(view_dict))

In [12]:
VIEW_FILE = 'view_dict_20191019.pkl'
with open(VIEW_FILE, 'wb') as handle:
    pickle.dump(view_dict, handle, protocol=pickle.HIGHEST_PROTOCOL)
print("saved view_dict as {}".format(VIEW_FILE))

saved view_dict as view_dict_20191019.pkl


In [9]:
print(len(view_dict))

9914556


In [ ]:
%%time
atb_dict = csv_event_rows_to_dict('pros_cart_item_addn_csv_120_pros_cart_item_addition.csv',chunksize=600000)
print(len(atb_dict))

In [ ]:
%%time
ATB_FILE = 'atb_dict_20191019.pkl'
with open(ATB_FILE, 'wb') as handle:
    pickle.dump(atb_dict, handle, protocol=pickle.HIGHEST_PROTOCOL)
print("saved purchase_dict as {}".format(ATB_FILE))

In [32]:
%%time
purchase_dict = csv_event_rows_to_dict('pros_cart_item_pur_csv_120_pros_cart_item_purchase.csv',chunksize=600000)
print(len(output_dict))

4291901
CPU times: user 1h 7min 52s, sys: 6min 24s, total: 1h 14min 17s
Wall time: 1h 14min 11s


In [13]:
PURCHASE_FILE = 'purchase_dict_20191019.pkl'
with open(PURCHASE_FILE, 'wb') as handle:
    pickle.dump(purchase_dict, handle, protocol=pickle.HIGHEST_PROTOCOL)
print("saved purchase_dict as {}".format(PURCHASE_FILE))

saved purchase_dict as view_dict_20191019.pkl


In [34]:
!ls *.pkl

atb_dict_20191019.pkl  purchase_dict_20191019.pkl  view_dict_20191019.pkl


In [40]:
purch_user_ids_min3 = [k for k,v in purchase_dict.items() if len(v) >= 3]
len(purch_user_ids_min3)

2093632

In [55]:
a_few_examples = list(purchase_dict.items())[:10]
a_few_examples

[('user_indiv_id', [('itemid', 'evnt_ts')]),
 ('-899320069214479257',
  [('75190', '2019-09-29 20:55:39'),
   ('75190', '2019-09-29 20:55:39'),
   ('2236949', '2019-09-29 20:55:39')]),
 ('1022976420', [('237110', '2019-07-24 17:38:23')]),
 ('331913352',
  [('286805', '2019-07-22 04:24:01'),
   ('286802', '2019-07-22 04:24:00'),
   ('1996291', '2019-07-22 04:24:01'),
   ('286803', '2019-08-02 13:31:12')]),
 ('198808388',
  [('296516', '2019-10-08 18:54:02'),
   ('2513452', '2019-07-06 19:47:19'),
   ('2226621', '2019-10-08 18:54:02'),
   ('5470096', '2019-10-08 18:54:02'),
   ('1205765', '2019-10-08 18:54:02'),
   ('1146116', '2019-10-08 18:54:02'),
   ('5520150', '2019-09-22 00:40:18'),
   ('2514421', '2019-07-06 19:47:19'),
   ('5461410', '2019-10-08 18:54:02'),
   ('9505285', '2019-10-08 18:54:02'),
   ('9502650', '2019-10-08 18:54:02'),
   ('9271126', '2019-10-08 18:54:02'),
   ('10083309', '2019-10-08 18:54:02'),
   ('338345', '2019-10-08 18:54:02'),
   ('9977612', '2019-10-08 18:5

### Get a full sequence

In [42]:
from operator import itemgetter
def show_sequences(user_id, show_data=False):
    # use pandas unique b/c maintains order
    vs = sorted(view_dict[user_id],key=itemgetter(1))
    vsu = pd.unique([x[0] for x in vs]).astype('str').tolist()
    at = sorted(atb_dict[user_id],key=itemgetter(1))
    atu = pd.unique([x[0] for x in at]).astype('str').tolist()
    pu = sorted(purchase_dict[user_id],key=itemgetter(1))
    puu = pd.unique([x[0] for x in pu]).astype('str').tolist()
    print("View Sequence")
    if show_data: print(vs)
    viz.display_recom_panel(vsu)
    print("Atb Sequence")
    if show_data: print(at)
    viz.display_recom_panel(atu)
    print("Pur Sequence")
    if show_data: print(pu)
    viz.display_recom_panel(puu)    

In [49]:
def show_interspersed_sequence(user_id, show_data=False):
    vs = sorted(view_dict[user_id],key=itemgetter(1))
    vsl = [x + ("view",) for x in vs]
    vsu = pd.unique([x[0] for x in vs]).astype('str').tolist()
    at = sorted(atb_dict[user_id],key=itemgetter(1))
    atl = [x + ("atb",) for x in at]
    atu = pd.unique([x[0] for x in at]).astype('str').tolist()
    pu = sorted(purchase_dict[user_id],key=itemgetter(1))
    pul = [x + ("purchase",) for x in pu]
    puu = pd.unique([x[0] for x in pu]).astype('str').tolist()
    full_sequence = sorted(vsl+atl+pul,key=itemgetter(1))
    past_status = ""
    current_status = ""
    current_items = []
    for item in full_sequence:
        if item[2] != current_status:
            if current_status and current_items:
                print(current_status)
                viz.display_recom_panel(current_items)
                current_items = []
                current_status = item[2]
            current_status = item[2]
            current_items.append(str(item[0]))        

In [52]:
rand_uid = np.random.choice(purch_user_ids_min3)
print(f"User id: {rand_uid}")
show_interspersed_sequence(rand_uid, show_data=True)

User id: 1038522723
view


$20.0 (8287018) 
 Pho Sho Me...

atb


$25.0 (7336979) 
 Columbia M...

view


...

$22.0 (3027306) 
 ELEMENT ME...

atb


$22.0 (2130625) 
 32 DEGREES...

purchase


$22.0 (2130625) 
 32 DEGREES...

In [56]:
rand_uid = np.random.choice(purch_user_ids_min3)
print(f"User id: {rand_uid}")
show_sequences(rand_uid, show_data=False)

User id: 28015624
View Sequence


$30.0 (402398) 
 SHISEIDO A... 
 
 $? (9371330) 
 ?... 
 
 $35.0 (527629) 
 LAURA MERC... 
 
 $16.5 (113762) 
 CLINIQUE S... 
 
 $50.0 (4781266) 
 MARTHA STE... 
 
 $36.0 (7135891) 
 Martha Ste... 
 
 $135.0 (7652701) 
 Martha Ste... 
 
 $49.99 (3053159) 
 MARTHA STE... 
 
 $12.5 (5358247) 
 TONYMOLY T... 
 
 $26.0 (322357) 
 JOCKEY HAL... 
 
 $89.25 (3605313) 
 T3 VOLUMIZ... 
 
 $495.0 (7910507) 
 Tommy Hilf... 
 
 $525.0 (7910512) 
 Tommy Hilf... 
 
 $89.0 (8286819) 
 Connected ... 
 
 $125.0 (7877680) 
 LAUREN RAL... 
 
 $495.0 (6271562) 
 Tommy Hilf... 
 
 $399.0 (7119000) 
 AppleWatc... 
 
 $495.0 (6271559) 
 Tommy Hilf... 
 
 $68.0 (7996898) 
 Free Peopl... 
 
 $58.0 (8505948) 
 Free Peopl... 
 
 $128.0 (8252591) 
 Free Peopl... 
 
 $148.0 (8316249) 
 Free Peopl... 
 
 $98.0 (8252533) 
 Free Peopl... 
 
 $128.0 (8067833) 
 Free Peopl... 
 
 $38.0 (7996847) 
 Free Peopl... 
 
 $98.0 (8505911) 
 Free Peopl... 
 
 $78.0 (8252570) 
 Free Peopl... 
 
 $148.0 (8067811) 
 Free Peopl... 
 
 $98.0 (8252574) 
 Free Peopl... 
 
 $98.0 (9345894) 
 Free Peopl... 
 
 $128.0 (8505903) 
 Free Peopl... 
 
 $48.0 (8316233) 
 Free Peopl... 
 
 $68.0 (8316095) 
 Free Peopl... 
 
 $138.0 (8924949) 
 Free Peopl... 
 
 $148.0 (9371198) 
 Free Peopl... 
 
 $148.0 (7996878) 
 Free Peopl... 
 
 $108.0 (9345908) 
 Free Peopl... 
 
 $108.0 (8924861) 
 Free Peopl... 
 
 $148.0 (8316124) 
 Free Peopl... 
 
 $58.0 (8252540) 
 Free Peopl... 
 
 $28.0 (8151281) 
 Free Peopl... 
 
 $138.0 (8924953) 
 Free Peopl... 
 
 $48.0 (5342080) 
 Free Peopl... 
 
 $165.0 (5238797) 
 Lauren Ral... 
 
 $25.0 (6749627) 
 Este Laud...

Atb Sequence


$30.0 (402398) 
 SHISEIDO A... 
 
 $495.0 (6271559) 
 Tommy Hilf...

Pur Sequence


$30.0 (402398) 
 SHISEIDO A... 
 
 $25.0 (4768568) 
 MARTHA STE... 
 
 $36.0 (7135891) 
 Martha Ste... 
 
 $99.99 (7060756) 
 Martha Ste... 
 
 $39.5 (8027542) 
 I.N.C. Gol... 
 
 $30.99 (4781270) 
 MARTHA STE... 
 
 $43.0 (224130) 
 MIRACLESUI... 
 
 $16.97 (958730) 
 STYLE & CO... 
 
 $16.5 (113762) 
 CLINIQUE S... 
 
 $54.5 (8226760) 
 I.N.C. Den...

In [37]:
show_sequences('1022976420')

View Sequence
[('237110', '2019-07-24 16:33:56'), ('237110', '2019-07-24 16:41:54'), ('237110', '2019-07-24 16:50:22'), ('237110', '2019-07-24 17:34:02'), ('1074650', '2019-07-24 17:43:22'), ('8031987', '2019-07-24 17:43:50'), ('589785', '2019-07-24 17:51:09'), ('237110', '2019-07-24 22:09:58'), ('9383247', '2019-07-25 01:38:02'), ('8060535', '2019-07-25 01:38:18'), ('7924980', '2019-07-25 01:39:56'), ('6822739', '2019-07-25 01:42:39'), ('6760138', '2019-07-25 01:44:30'), ('7722384', '2019-07-25 01:48:35'), ('8826049', '2019-07-25 01:48:44'), ('7449740', '2019-07-25 01:49:05'), ('6994349', '2019-07-25 01:50:09'), ('8826049', '2019-07-25 01:51:07'), ('6994349', '2019-07-25 01:51:31'), ('7722384', '2019-07-25 01:51:58'), ('5882481', '2019-07-25 01:53:16'), ('8907585', '2019-07-25 02:03:51'), ('8505520', '2019-07-25 02:04:01'), ('7080226', '2019-07-25 02:07:21'), ('7722384', '2019-07-25 02:07:43'), ('8826049', '2019-07-25 02:08:39'), ('8826050', '2019-07-25 02:09:12'), ('8906302', '2019-0

$120.0 (237110) 
 ANGEL BY M... 
 
 $3807.0 (1074650) 
 AILEY KING... 
 
 $1199.0 (8031987) 
 Virgil Kin... 
 
 $11.99 (589785) 
 MELISSA AN... 
 
 $75.0 (9383247) 
 Fila Women... 
 
 $65.0 (8060535) 
 Fila Women... 
 
 $65.0 (7924980) 
 Fila Women... 
 
 $66.0 (6822739) 
 Fila Rosly... 
 
 $25.0 (6760138) 
 adidas Lin... 
 
 $50.0 (7722384) 
 Fila Boys'... 
 
 $55.0 (8826049) 
 Fila Boys'... 
 
 $55.0 (7449740) 
 Fila Littl... 
 
 $65.0 (6994349) 
 Fila Men's... 
 
 $65.0 (5882481) 
 Fila Const... 
 
 $70.0 (8907585) 
 Nike Littl... 
 
 $50.0 (8505520) 
 Champion L... 
 
 $12.0 (7080226) 
 Champion L... 
 
 $55.0 (8826050) 
 Fila Boys'... 
 
 $65.0 (8906302) 
 Fila Boys'... 
 
 $43.0 (9926957) 
 Nike Toddl... 
 
 $48.0 (9885096) 
 Nike Toddl... 
 
 $80.0 (8346956) 
 adidas Boy... 
 
 $40.0 (8998319) 
 Fila Toddl... 
 
 $35.0 (6920361) 
 bennet tee... 
 
 $45.0 (6822750) 
 Fila Virgi... 
 
 $38.0 (2760799) 
 FILA TIONN... 
 
 $32.0 (6142982) 
 adidas Big... 
 
 $36.0 (6813849) 
 adidas Bab... 
 
 $54.0 (5728879) 
 adidas 2-P... 
 
 $30.0 (6142935) 
 adidas Lit... 
 
 $75.0 (5859885) 
 adidas Spa... 
 
 $20.0 (8064551) 
 adidas Big... 
 
 $65.0 (2582051) 
 ADIDAS KID... 
 
 $50.0 (7262503) 
 Fila Boys'...

Atb Sequence
[('237110', '2019-07-24 16:34:20'), ('237110', '2019-07-24 16:40:55'), ('237110', '2019-07-24 16:41:59'), ('237110', '2019-07-24 16:42:06'), ('237110', '2019-07-24 17:34:40'), ('237110', '2019-07-24 17:34:57'), ('237110', '2019-07-24 17:38:23'), ('6760138', '2019-07-25 01:44:42'), ('6760138', '2019-07-25 02:18:59')]


$120.0 (237110) 
 ANGEL BY M... 
 
 $25.0 (6760138) 
 adidas Lin...

Pur Sequence
[('237110', '2019-07-24 17:38:23')]


$120.0 (237110) 
 ANGEL BY M...

In [38]:
show_sequences('142350418')

View Sequence
[('8550627', '2019-07-02 20:21:43'), ('6784804', '2019-07-02 20:23:46'), ('8259701', '2019-07-02 20:23:52'), ('8259701', '2019-07-03 01:04:26'), ('178703', '2019-08-25 00:42:20'), ('683245', '2019-08-25 00:45:04'), ('296516', '2019-08-25 00:48:28'), ('2127504', '2019-08-25 00:52:25'), ('6952716', '2019-08-25 18:39:25'), ('4806798', '2019-08-25 18:40:13'), ('644915', '2019-08-25 18:40:28'), ('4806798', '2019-08-25 18:40:57'), ('683245', '2019-08-25 18:41:33'), ('296516', '2019-08-25 18:49:55'), ('2142453', '2019-08-25 18:53:57'), ('6283428', '2019-08-28 02:04:49'), ('4436825', '2019-08-28 02:07:00'), ('9510448', '2019-08-28 02:10:27'), ('75134', '2019-08-28 02:19:22'), ('9595679', '2019-08-28 12:51:08'), ('9595679', '2019-08-29 21:24:56'), ('296516', '2019-09-02 04:03:23'), ('683245', '2019-09-02 04:05:23'), ('683245', '2019-09-08 15:10:28'), ('683245', '2019-09-08 15:13:47'), ('683245', '2019-09-08 15:14:17'), ('683245', '2019-09-08 15:14:26'), ('9338585', '2019-10-09 19:

$50.0 (8550627) 
 Skechers W... 
 
 $50.0 (6784804) 
 Skechers W... 
 
 $50.0 (8259701) 
 Skechers W... 
 
 $20.0 (178703) 
 LANCME CO... 
 
 $19.0 (683245) 
 URBAN DECA... 
 
 $20.0 (296516) 
 URBAN DECA... 
 
 $20.0 (2127504) 
 URBAN DECA... 
 
 $49.0 (6952716) 
 Urban Deca... 
 
 $54.0 (4806798) 
 URBAN DECA... 
 
 $54.0 (644915) 
 URBAN DECA... 
 
 $47.99 (2142453) 
 URBAN DECA... 
 
 $34.0 (6283428) 
 Urban Deca... 
 
 $34.0 (4436825) 
 URBAN DECA... 
 
 $39.0 (9510448) 
 Urban Deca... 
 
 $17.0 (75134) 
 CLINIQUE 3... 
 
 $? (9595679) 
 ?... 
 
 $? (9338585) 
 ?... 
 
 $59.5 (8982043) 
 Charter Cl... 
 
 $? (10209602) 
 ?... 
 
 $27.5 (116214) 
 LANCME HY... 
 
 $24.5 (2914388) 
 CLINIQUE C... 
 
 $16.0 (253779) 
 CLINIQUE A... 
 
 $15.5 (258770) 
 CLINIQUE A... 
 
 $49.0 (6373604) 
 Urban Deca... 
 
 $24.0 (2839995) 
 URBAN DECA... 
 
 $26.0 (2839987) 
 URBAN DECA... 
 
 $26.0 (2840102) 
 URBAN DECA... 
 
 $24.0 (2839994) 
 URBAN DECA... 
 
 $45.0 (2840098) 
 URBAN DECA... 
 
 $32.0 (2840026) 
 URBAN DECA... 
 
 $26.0 (2839990) 
 URBAN DECA...

Atb Sequence
[('153337', '2019-08-02 01:47:05'), ('8505511', '2019-08-02 01:49:20'), ('153337', '2019-08-02 01:50:37'), ('8505511', '2019-08-02 01:50:37'), ('153337', '2019-08-02 01:52:18'), ('683245', '2019-08-25 00:48:05'), ('296516', '2019-08-25 00:49:59'), ('2127504', '2019-08-25 00:52:52'), ('683245', '2019-08-25 18:45:00'), ('683245', '2019-08-25 18:46:29'), ('683245', '2019-08-25 18:49:22'), ('296516', '2019-08-25 18:52:50'), ('6283428', '2019-08-28 02:06:24'), ('4436825', '2019-08-28 02:09:06'), ('9510448', '2019-08-28 02:14:21'), ('75134', '2019-08-28 02:19:54'), ('9595679', '2019-08-28 12:53:18'), ('9595679', '2019-08-29 21:25:16'), ('296516', '2019-09-02 04:03:38'), ('296516', '2019-09-02 04:04:31'), ('683245', '2019-09-02 04:06:44'), ('9338585', '2019-10-09 19:39:19'), ('9338585', '2019-10-09 19:39:35'), ('116214', '2019-10-09 19:42:41'), ('253779', '2019-10-09 19:50:20'), ('258770', '2019-10-09 19:53:34'), ('6373604', '2019-10-14 17:55:53'), ('2840102', '2019-10-14 18:03:0

$27.5 (153337) 
 LANCME HY... 
 
 $55.0 (8505511) 
 Skechers W... 
 
 $19.0 (683245) 
 URBAN DECA... 
 
 $20.0 (296516) 
 URBAN DECA... 
 
 $20.0 (2127504) 
 URBAN DECA... 
 
 $34.0 (6283428) 
 Urban Deca... 
 
 $34.0 (4436825) 
 URBAN DECA... 
 
 $39.0 (9510448) 
 Urban Deca... 
 
 $17.0 (75134) 
 CLINIQUE 3... 
 
 $? (9595679) 
 ?... 
 
 $? (9338585) 
 ?... 
 
 $27.5 (116214) 
 LANCME HY... 
 
 $16.0 (253779) 
 CLINIQUE A... 
 
 $15.5 (258770) 
 CLINIQUE A... 
 
 $49.0 (6373604) 
 Urban Deca... 
 
 $26.0 (2840102) 
 URBAN DECA... 
 
 $32.0 (2840026) 
 URBAN DECA...

Pur Sequence
[('153337', '2019-08-02 01:52:18'), ('296516', '2019-08-25 00:57:33'), ('2127504', '2019-08-25 00:57:33'), ('683245', '2019-08-25 00:57:33'), ('296516', '2019-08-25 18:58:57'), ('683245', '2019-08-25 18:58:57'), ('9510448', '2019-08-28 02:25:03'), ('4436825', '2019-08-28 02:25:03'), ('6283428', '2019-08-28 02:25:03'), ('75134', '2019-08-28 02:25:03'), ('9595679', '2019-08-29 21:27:14'), ('683245', '2019-09-02 04:10:09'), ('296516', '2019-09-02 04:10:09'), ('253779', '2019-10-09 19:57:13'), ('258770', '2019-10-09 19:57:13'), ('9338585', '2019-10-09 19:57:13'), ('116214', '2019-10-09 19:57:13'), ('6373604', '2019-10-14 18:07:55'), ('2840026', '2019-10-14 18:07:55'), ('2840102', '2019-10-14 18:07:55'), ('6373604', '2019-10-15 02:30:40')]


$27.5 (153337) 
 LANCME HY... 
 
 $20.0 (296516) 
 URBAN DECA... 
 
 $20.0 (2127504) 
 URBAN DECA... 
 
 $19.0 (683245) 
 URBAN DECA... 
 
 $39.0 (9510448) 
 Urban Deca... 
 
 $34.0 (4436825) 
 URBAN DECA... 
 
 $34.0 (6283428) 
 Urban Deca... 
 
 $17.0 (75134) 
 CLINIQUE 3... 
 
 $? (9595679) 
 ?... 
 
 $16.0 (253779) 
 CLINIQUE A... 
 
 $15.5 (258770) 
 CLINIQUE A... 
 
 $? (9338585) 
 ?... 
 
 $27.5 (116214) 
 LANCME HY... 
 
 $49.0 (6373604) 
 Urban Deca... 
 
 $32.0 (2840026) 
 URBAN DECA... 
 
 $26.0 (2840102) 
 URBAN DECA...

In [21]:
example_sequence = view_dict['1022976420'] + atb_dict['1022976420'] + output_dict['1022976420']


In [33]:
unique_pids_sequence = np.unique(list(zip(*example_sequence))[0])
unique_pids_sequence

array(['1074650', '237110', '2582051', '2760799', '5728879', '5859885',
       '5882481', '589785', '6142935', '6142982', '6760138', '6813849',
       '6822739', '6822750', '6920361', '6994349', '7080226', '7262503',
       '7449740', '7722384', '7924980', '8031987', '8060535', '8064551',
       '8346956', '8505520', '8826049', '8826050', '8906302', '8907585',
       '8998319', '9383247', '9885096', '9926957'], dtype='<U7')

In [31]:
viz.display_recom_panel(unique_pids_sequence)

$3807.0 (1074650) 
 AILEY KING... 
 
 $120.0 (237110) 
 ANGEL BY M... 
 
 $65.0 (2582051) 
 ADIDAS KID... 
 
 $38.0 (2760799) 
 FILA TIONN... 
 
 $54.0 (5728879) 
 adidas 2-P... 
 
 $75.0 (5859885) 
 adidas Spa... 
 
 $65.0 (5882481) 
 Fila Const... 
 
 $11.99 (589785) 
 MELISSA AN... 
 
 $30.0 (6142935) 
 adidas Lit... 
 
 $32.0 (6142982) 
 adidas Big... 
 
 $25.0 (6760138) 
 adidas Lin... 
 
 $36.0 (6813849) 
 adidas Bab... 
 
 $66.0 (6822739) 
 Fila Rosly... 
 
 $45.0 (6822750) 
 Fila Virgi... 
 
 $35.0 (6920361) 
 bennet tee... 
 
 $65.0 (6994349) 
 Fila Men's... 
 
 $12.0 (7080226) 
 Champion L... 
 
 $50.0 (7262503) 
 Fila Boys'... 
 
 $55.0 (7449740) 
 Fila Littl... 
 
 $50.0 (7722384) 
 Fila Boys'... 
 
 $65.0 (7924980) 
 Fila Women... 
 
 $1199.0 (8031987) 
 Virgil Kin... 
 
 $65.0 (8060535) 
 Fila Women... 
 
 $20.0 (8064551) 
 adidas Big... 
 
 $80.0 (8346956) 
 adidas Boy... 
 
 $50.0 (8505520) 
 Champion L... 
 
 $55.0 (8826049) 
 Fila Boys'... 
 
 $55.0 (8826050) 
 Fila Boys'... 
 
 $65.0 (8906302) 
 Fila Boys'... 
 
 $70.0 (8907585) 
 Nike Littl... 
 
 $40.0 (8998319) 
 Fila Toddl... 
 
 $75.0 (9383247) 
 Fila Women... 
 
 $48.0 (9885096) 
 Nike Toddl... 
 
 $43.0 (9926957) 
 Nike Toddl...

In [39]:
show_sequences('41872816')

View Sequence
[('4645900', '2019-08-10 22:42:26'), ('4504737', '2019-08-10 22:42:54'), ('2530348', '2019-08-10 22:44:51'), ('4888352', '2019-08-10 22:48:29'), ('6672465', '2019-08-10 22:51:54'), ('4504737', '2019-08-10 22:53:36'), ('4504737', '2019-08-10 22:54:28'), ('1607181', '2019-08-10 22:55:35'), ('4645900', '2019-08-10 22:56:22'), ('6596183', '2019-08-10 22:57:48'), ('4815268', '2019-08-10 22:59:01'), ('750734', '2019-08-10 23:05:09'), ('6596183', '2019-08-10 23:07:03'), ('6596183', '2019-08-10 23:08:53'), ('927595', '2019-08-10 23:11:32'), ('574131', '2019-08-10 23:12:54'), ('574131', '2019-08-10 23:13:47'), ('667241', '2019-08-10 23:18:25'), ('574131', '2019-08-10 23:23:45'), ('6596183', '2019-08-10 23:23:59'), ('574131', '2019-08-10 23:25:50'), ('6596183', '2019-08-10 23:27:00'), ('5599871', '2019-08-10 23:30:40'), ('6596183', '2019-08-10 23:40:44'), ('790724', '2019-08-10 23:42:01'), ('5793386', '2019-08-10 23:43:56'), ('750734', '2019-08-10 23:50:54'), ('296516', '2019-08-10

$nan (4645900) 
 AQ TEXTILE... 
 
 $nan (4504737) 
 CHARTER CL... 
 
 $nan (2530348) 
 MICROFIBER... 
 
 $nan (4888352) 
 MADISON PA... 
 
 $? (6672465) 
 ?... 
 
 $32.0 (1607181) 
 HOTEL COLL... 
 
 $42.0 (6596183) 
 Maidenform... 
 
 $21.99 (4815268) 
 MAIDENFORM... 
 
 $42.0 (750734) 
 MAIDENFORM... 
 
 $42.0 (927595) 
 MAIDENFORM... 
 
 $36.0 (574131) 
 MAIDENFORM... 
 
 $38.0 (667241) 
 MAIDENFORM... 
 
 $42.0 (5599871) 
 Maidenform... 
 
 $42.0 (790724) 
 MAIDENFORM... 
 
 $40.0 (5793386) 
 Warner's P... 
 
 $20.0 (296516) 
 URBAN DECA... 
 
 $? (9534364) 
 ?... 
 
 $? (9534317) 
 ?... 
 
 $nan (4796247) 
 FIESTA 3-P... 
 
 $47.0 (5594509) 
 Fiesta 3-P... 
 
 $47.0 (5594545) 
 Fiesta 3-P... 
 
 $47.0 (5594422) 
 Fiesta 3-P... 
 
 $47.0 (9218210) 
 Fiesta Mea... 
 
 $47.0 (5594434) 
 Fiesta 3-P... 
 
 $47.0 (5428264) 
 Fiesta 3-P... 
 
 $47.0 (5594429) 
 Fiesta 3-P... 
 
 $? (9534432) 
 ?... 
 
 $125.0 (9021716) 
 Martha Ste... 
 
 $125.0 (9021717) 
 Martha Ste... 
 
 $53.0 (223134) 
 THE CELLAR... 
 
 $43.0 (2258673) 
 THE CELLAR... 
 
 $23.99 (3097783) 
 THE CELLAR...

Atb Sequence
[('750734', '2019-08-10 23:06:27'), ('6596183', '2019-08-10 23:07:17'), ('6596183', '2019-08-10 23:07:27'), ('574131', '2019-08-10 23:13:59'), ('574131', '2019-08-10 23:14:07'), ('574131', '2019-08-10 23:17:16'), ('667241', '2019-08-10 23:21:54'), ('667241', '2019-08-10 23:22:18'), ('5599871', '2019-08-10 23:31:03'), ('6596183', '2019-08-10 23:41:03'), ('790724', '2019-08-10 23:42:17'), ('5793386', '2019-08-10 23:44:32'), ('296516', '2019-08-10 23:58:18'), ('4795349', '2019-08-11 00:13:07'), ('5594509', '2019-08-11 00:13:30'), ('5594545', '2019-08-11 00:13:46'), ('5594422', '2019-08-11 00:14:58'), ('9218210', '2019-08-11 00:16:01'), ('5594434', '2019-08-11 00:16:20'), ('5428264', '2019-08-11 00:20:49'), ('5594429', '2019-08-11 00:22:34'), ('9021716', '2019-08-11 00:32:29'), ('223134', '2019-08-11 00:38:04')]


$42.0 (750734) 
 MAIDENFORM... 
 
 $42.0 (6596183) 
 Maidenform... 
 
 $36.0 (574131) 
 MAIDENFORM... 
 
 $38.0 (667241) 
 MAIDENFORM... 
 
 $42.0 (5599871) 
 Maidenform... 
 
 $42.0 (790724) 
 MAIDENFORM... 
 
 $40.0 (5793386) 
 Warner's P... 
 
 $20.0 (296516) 
 URBAN DECA... 
 
 $47.0 (4795349) 
 FIESTA POP... 
 
 $47.0 (5594509) 
 Fiesta 3-P... 
 
 $47.0 (5594545) 
 Fiesta 3-P... 
 
 $47.0 (5594422) 
 Fiesta 3-P... 
 
 $47.0 (9218210) 
 Fiesta Mea... 
 
 $47.0 (5594434) 
 Fiesta 3-P... 
 
 $47.0 (5428264) 
 Fiesta 3-P... 
 
 $47.0 (5594429) 
 Fiesta 3-P... 
 
 $125.0 (9021716) 
 Martha Ste... 
 
 $53.0 (223134) 
 THE CELLAR...

Pur Sequence
[('790724', '2019-08-10 23:56:05'), ('574131', '2019-08-10 23:56:05'), ('667241', '2019-08-10 23:56:05'), ('5793386', '2019-08-10 23:56:05'), ('6596183', '2019-08-10 23:56:05'), ('5599871', '2019-08-10 23:56:05'), ('750734', '2019-08-10 23:56:05'), ('296516', '2019-08-11 00:44:13'), ('5594429', '2019-08-11 00:44:13'), ('4795349', '2019-08-11 00:44:13'), ('5594434', '2019-08-11 00:44:13'), ('5594509', '2019-08-11 00:44:13'), ('5594422', '2019-08-11 00:44:13'), ('5594545', '2019-08-11 00:44:13'), ('9218210', '2019-08-11 00:44:13'), ('9021716', '2019-08-11 00:44:13'), ('5428264', '2019-08-11 00:44:13'), ('223134', '2019-08-11 00:44:13')]


$42.0 (790724) 
 MAIDENFORM... 
 
 $36.0 (574131) 
 MAIDENFORM... 
 
 $38.0 (667241) 
 MAIDENFORM... 
 
 $40.0 (5793386) 
 Warner's P... 
 
 $42.0 (6596183) 
 Maidenform... 
 
 $42.0 (5599871) 
 Maidenform... 
 
 $42.0 (750734) 
 MAIDENFORM... 
 
 $20.0 (296516) 
 URBAN DECA... 
 
 $47.0 (5594429) 
 Fiesta 3-P... 
 
 $47.0 (4795349) 
 FIESTA POP... 
 
 $47.0 (5594434) 
 Fiesta 3-P... 
 
 $47.0 (5594509) 
 Fiesta 3-P... 
 
 $47.0 (5594422) 
 Fiesta 3-P... 
 
 $47.0 (5594545) 
 Fiesta 3-P... 
 
 $47.0 (9218210) 
 Fiesta Mea... 
 
 $125.0 (9021716) 
 Martha Ste... 
 
 $47.0 (5428264) 
 Fiesta 3-P... 
 
 $53.0 (223134) 
 THE CELLAR...

In [6]:
a_few=list(output_dict.items())[:1000]
a_few[:10]

[('user_indiv_id', [('itemid', 'evnt_ts')]),
 ('-899320069214479257',
  [('75190', '2019-09-29 20:55:39'),
   ('75190', '2019-09-29 20:55:39'),
   ('2236949', '2019-09-29 20:55:39')]),
 ('1022976420', [('237110', '2019-07-24 17:38:23')]),
 ('331913352',
  [('286805', '2019-07-22 04:24:01'),
   ('286802', '2019-07-22 04:24:00'),
   ('1996291', '2019-07-22 04:24:01'),
   ('286803', '2019-08-02 13:31:12')]),
 ('198808388',
  [('296516', '2019-10-08 18:54:02'),
   ('2513452', '2019-07-06 19:47:19'),
   ('2226621', '2019-10-08 18:54:02'),
   ('5470096', '2019-10-08 18:54:02'),
   ('1205765', '2019-10-08 18:54:02'),
   ('1146116', '2019-10-08 18:54:02'),
   ('5520150', '2019-09-22 00:40:18'),
   ('2514421', '2019-07-06 19:47:19'),
   ('5461410', '2019-10-08 18:54:02'),
   ('9505285', '2019-10-08 18:54:02'),
   ('9502650', '2019-10-08 18:54:02'),
   ('9271126', '2019-10-08 18:54:02'),
   ('10083309', '2019-10-08 18:54:02'),
   ('338345', '2019-10-08 18:54:02'),
   ('9977612', '2019-10-08 18:5

In [83]:
view_rows = get_csv_rows('pros_prod_vw_csv_120_pros_product_view.csv',numcols=4)
print(len(view_rows))
view_rows[:3]

375938096


[['user_indiv_id', 'itemid', 'evnt_ts', 'sessionid'],
 ['161322486',
  '1806',
  '2019-06-26 17:46:22',
  '57133676298705600281061543070534137125503'],
 ['123152321',
  '64887',
  '2019-08-04 11:05:37',
  '756201711834091504050373584764359394489975620171183409150405037358476435939448']]

In [ ]:
view_cols = view_rows[0]
view_rows = pd.DataFrame(view_rows)
view_rows.columns = view_cols
view_rows.head()

In [1]:
view_rows[:3]

NameError: name 'view_rows' is not defined

In [65]:
#test = pd.read_csv('results-20191022-152105.csv')
test = pd.read_csv('pros_cart_item_addn_csv_pros_cart_item_addition.csv',sep='\t')
print(test.shape)
test.head()

(33046185, 4)


,user_indiv_id,itemid,evnt_ts,sessionid
0,23366872,237110,2019-08-25 12:10:03,9121353398468207352992220960641855277399121353...
1,167404204,106344,2019-10-17 03:31:25,8624261677556333870120113964743851738686242616...
2,1026487863,106344,2019-09-21 19:24:38,3262225133254954180461170530068461626474326222...
3,78588651,106344,2019-09-15 13:32:52,3342735317420968053691753015399636163211334273...
4,150977898,401011,2019-10-06 16:35:37,1696738881285883358215043740839664615810169673...


In [68]:
min(test['evnt_ts']), max(test['evnt_ts'])

('2019-08-20 00:00:00', '2019-10-18 23:59:59')

In [69]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 33046185 entries, 0 to 33046184
Data columns (total 4 columns):
user_indiv_id    int64
itemid           int64
evnt_ts          object
sessionid        object
dtypes: int64(2), object(2)
memory usage: 1008.5+ MB


### To create a sequence of events we must:
* Group events by "vstr_id" or "user_indiv_id"
* Then sort by "evnt_ts"

In [70]:
sort1 = test.sort_values('evnt_ts')
sort1.head()

,user_indiv_id,itemid,evnt_ts,sessionid
13247240,112338792,9996280,2019-08-20 00:00:00,3297094789467973269691753025063303248617332970...
31075855,79323285,4648754,2019-08-20 00:00:01,8785495632687523678137183020366777391714187854...
18355531,120991693,1356233,2019-08-20 00:00:01,2099095209120684436434554810500908238517320990...
25503263,259798304,8315784,2019-08-20 00:00:01,9148496090015697266778932720021728587011914849...
9099149,44351390,9618868,2019-08-20 00:00:01,3328369169359421189691753027479225712573328369...


In [71]:
sort2 = test.copy()
sort2['evnt_ts'] = pd.to_datetime(sort2['evnt_ts'])
sort2 = sort2.sort_values('evnt_ts')
sort2.head()

,user_indiv_id,itemid,evnt_ts,sessionid
13247240,112338792,9996280,2019-08-20 00:00:00,3297094789467973269691753025063303248617332970...
31075855,79323285,4648754,2019-08-20 00:00:01,8785495632687523678137183020366777391714187854...
18355531,120991693,1356233,2019-08-20 00:00:01,2099095209120684436434554810500908238517320990...
25503263,259798304,8315784,2019-08-20 00:00:01,9148496090015697266778932720021728587011914849...
9099149,44351390,9618868,2019-08-20 00:00:01,3328369169359421189691753027479225712573328369...


In [73]:
#sort2.groupby(["vstr_id"]).apply(lambda x: x.sort_values(["evnt_ts"])).reset_index(drop=True)
grouptest2 = sort2.groupby(["user_indiv_id"]).apply(lambda x: x.sort_values(["evnt_ts"])).reset_index(drop=True)
grouptest2.head()

,user_indiv_id,itemid,evnt_ts,sessionid
0,-9223346256872843933,404141,2019-09-15 02:57:21,3542632177959033695706765902810329180713542632...
1,-9223346256872843933,585676,2019-09-18 22:14:21,3542632177959033695706765902810329180723542632...
2,-9223346256872843933,10216247,2019-09-27 22:37:25,3542632177959033695706765902810329180733542632...
3,-9223346256872843933,8336650,2019-09-27 22:39:03,3542632177959033695706765902810329180733542632...
4,-9223346256872843933,7975690,2019-09-27 22:40:52,3542632177959033695706765902810329180733542632...


In [75]:
user1 = grouptest2.query('user_indiv_id == -9223346256872843933')
user1

,user_indiv_id,itemid,evnt_ts,sessionid
0,-9223346256872843933,404141,2019-09-15 02:57:21,3542632177959033695706765902810329180713542632...
1,-9223346256872843933,585676,2019-09-18 22:14:21,3542632177959033695706765902810329180723542632...
2,-9223346256872843933,10216247,2019-09-27 22:37:25,3542632177959033695706765902810329180733542632...
3,-9223346256872843933,8336650,2019-09-27 22:39:03,3542632177959033695706765902810329180733542632...
4,-9223346256872843933,7975690,2019-09-27 22:40:52,3542632177959033695706765902810329180733542632...


In [78]:
viz.display_recom_panel([str(x) for x in user1['itemid']])

$60.0 (404141) 
 THOMSON PO... 
 
 $20.0 (585676) 
 GODINGER C... 
 
 $? (10216247) 
 ?... 
 
 $36.5 (8336650) 
 Karen Scot... 
 
 $64.5 (7975690) 
 Charter Cl...

In [80]:
grouptest2.shape

(33046185, 4)